In [13]:
import requests
import csv
import time
from typing import List, Dict

# ==============================
# 설정
# ==============================
API_KEY = "E5ERXYC8Q7NMPXF39D32D6ZNXVWKNXV9NQ"  # 실제 키로 교체
CONTRACT_ADDRESS = "0x6b175474e89094c44da98b954eedeac495271d0f"  # DAI
TARGET_TRANSFERS = 100_000
PAGE_SIZE = 10_000  # Etherscan 최대 offset
REQUEST_DELAY = 0.25  # 1초에 4회 요청 (무료 키 기준 안전)
CSV_FILENAME = "dai_transfers.csv"
CHAIN_ID = 1  # Ethereum 메인넷

# ==============================
# 헬퍼 함수
# ==============================
def wei_to_ether(wei: str, decimals: int = 18) -> float:
    """wei 단위를 ether 단위로 변환 (DAI는 18 decimals)"""
    return int(wei) / (10 ** decimals)

def fetch_page(page: int) -> List[Dict]:
    """한 페이지를 API로 요청 (V2 호환)"""
    url = (
        f"https://api.etherscan.io/v2/api?"  # V2 베이스 URL
        f"chainid={CHAIN_ID}&"  # 새로 추가: 체인 ID
        f"module=account&action=tokentx"
        f"&contractaddress={CONTRACT_ADDRESS}"
        f"&page={page}&offset={PAGE_SIZE}"
        f"&sort=desc&apikey={API_KEY}"
    )
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        if data.get("status") == "0":
            print(f"API 오류 (페이지 {page}): {data.get('message')} - {data.get('result')}")
            return []
        return data.get("result", [])
    except requests.exceptions.RequestException as e:
        print(f"요청 실패 (페이지 {page}): {e}")
        return []

# ==============================
# 메인 로직
# ==============================
def main():
    all_transfers = []
    page = 1

    print(f"DAI 전송 내역 수집 시작... 목표: {TARGET_TRANSFERS}건")

    while len(all_transfers) < TARGET_TRANSFERS:
        print(f"페이지 {page} 요청 중... (현재 {len(all_transfers)}건 수집)")
        transfers = fetch_page(page)

        if not transfers:
            print("더 이상 데이터 없음 또는 API 제한 도달.")
            break

        all_transfers.extend(transfers)
        page += 1
        time.sleep(REQUEST_DELAY)  # Rate limit 방지

        # 목표 도달 시 조기 종료
        if len(all_transfers) >= TARGET_TRANSFERS:
            all_transfers = all_transfers[:TARGET_TRANSFERS]
            break

    # ==============================
    # 모든 필드 정의 (Etherscan V2 기준)
    # ==============================
    fieldnames = [
        'blockNumber', 'timeStamp', 'hash', 'nonce', 'blockHash',
        'from', 'contractAddress', 'to', 'value', 'valueDAI',
        'tokenName', 'tokenSymbol', 'tokenDecimal',
        'transactionIndex', 'gas', 'gasPrice', 'gasUsed',
        'cumulativeGasUsed', 'input', 'confirmations'
    ]

    print(f"\n총 {len(all_transfers):,}건 수집 완료. CSV 저장 중...")
    with open(CSV_FILENAME, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()

        for tx in all_transfers:
            # value를 DAI 단위로 변환 (안전하게 처리)
            try:
                decimals = int(tx.get('tokenDecimal', 18))
                value_dai = wei_to_ether(tx.get('value', '0'), decimals)
            except:
                value_dai = 0.0

            writer.writerow({
                'blockNumber': tx.get('blockNumber'),
                'timeStamp': tx.get('timeStamp'),
                'hash': tx.get('hash'),
                'nonce': tx.get('nonce'),
                'blockHash': tx.get('blockHash'),
                'from': tx.get('from'),
                'contractAddress': tx.get('contractAddress'),
                'to': tx.get('to'),
                'value': tx.get('value'),
                'valueDAI': f"{value_dai:.18f}".rstrip('0').rstrip('.') if '.' in f"{value_dai:.18f}" else f"{value_dai:.18f}",
                'tokenName': tx.get('tokenName'),
                'tokenSymbol': tx.get('tokenSymbol'),
                'tokenDecimal': tx.get('tokenDecimal'),
                'transactionIndex': tx.get('transactionIndex'),
                'gas': tx.get('gas'),
                'gasPrice': tx.get('gasPrice'),
                'gasUsed': tx.get('gasUsed'),
                'cumulativeGasUsed': tx.get('cumulativeGasUsed'),
                'input': tx.get('input'),
                'confirmations': tx.get('confirmations')
            })

    print(f"CSV 저장 완료: {CSV_FILENAME} ({len(all_transfers):,}건)")

if __name__ == "__main__":
    main()

DAI 전송 내역 수집 시작... 목표: 100000건
페이지 1 요청 중... (현재 0건 수집)
페이지 2 요청 중... (현재 10000건 수집)
API 오류 (페이지 2): Result window is too large, PageNo x Offset size must be less than or equal to 10000 - None
더 이상 데이터 없음 또는 API 제한 도달.

총 10,000건 수집 완료. CSV 저장 중...
CSV 저장 완료: dai_transfers.csv (10,000건)
